# **Microsoft Defender Threat Intelligence**

## Jupyter NoteBook Demo for GBBs to present MDTI Graph API

##### Microsoft Defender Threat Intelligence (MDTI) is a complete threat intelligence platform that enables security professionals to ingest, analyze and act upon trillions of signals collected from across the internet and processed by security experts and AI. MDTI allows users to uncover and understand the global threat landscape as it relates to their organization, including context around vulnerabilities, threat actors, and the threat infrastructure that might be used to attack them.

##### Microsoft processes over 65-trillion signals every day. The key to operationalizing threat intelligence is to make it not just applicable, but actionable. Combined with AI and the expertise of 8,500 dedicated security professionals, this wealth of external telemetry and signal data is wrangled into immediately actionable insights.

#  What Data can you obtain using the MDTI Graph API:
- Reputation Scoring. Dynamically calculated severity scoring for IP addresses, domains, and hosts based on real-world threats and observations.
- Components
- Trackers
- Threat Intel Article
- Intel Profiles
- Vulnerability

_____________________________________________________________
# Get a Token from Azure Active Directory


#### Get the Client Secret and set to an environment variable

In [2]:
import os

# Read the client secret from a text file
with open("client_secret.txt", "r") as f:
    client_secret = f.read().strip()

# Set the client secret as an environment variable
os.environ["CLIENT_SECRET"] = client_secret

#### Get the Token with this cell

In [3]:
from msal import ConfidentialClientApplication

# Azure AD application credentials
client_id = "30232d95-233d-403b-a7d6-4aef57439f72"
tenant_id = "4b2462a4-bbee-495a-a0e1-f23ae524cc9c"


# Create a ConfidentialClientApplication object
app = ConfidentialClientApplication(
    client_id=client_id,
    client_credential=client_secret,
    authority=f"https://login.microsoftonline.com/{tenant_id}",
)

# Get a token from Azure AD
result = None
scopes = ["https://graph.microsoft.com/.default"]
result = app.acquire_token_silent(scopes=scopes, account=None)

if not result:
    result = app.acquire_token_for_client(scopes=scopes)

# Get the access token
access_token = result["access_token"]

# Print the access token
print("Access Token:", access_token)


Access Token: eyJ0eXAiOiJKV1QiLCJub25jZSI6Im1XRGo5MmJzdlI4aGhFZjZabTNiempBcW5oaEVKcjBTS3BnMnlYc3h3cTgiLCJhbGciOiJSUzI1NiIsIng1dCI6Ii1LSTNROW5OUjdiUm9meG1lWm9YcWJIWkdldyIsImtpZCI6Ii1LSTNROW5OUjdiUm9meG1lWm9YcWJIWkdldyJ9.eyJhdWQiOiJodHRwczovL2dyYXBoLm1pY3Jvc29mdC5jb20iLCJpc3MiOiJodHRwczovL3N0cy53aW5kb3dzLm5ldC80YjI0NjJhNC1iYmVlLTQ5NWEtYTBlMS1mMjNhZTUyNGNjOWMvIiwiaWF0IjoxNjgxNDc3ODA4LCJuYmYiOjE2ODE0Nzc4MDgsImV4cCI6MTY4MTQ4MTcwOCwiYWlvIjoiRTJaZ1lHZzFpSzc0RTNLTE9aVHQ2TGRLMjRhSEFBPT0iLCJhcHBfZGlzcGxheW5hbWUiOiJNRFRJLUFQSS1BUFAtRmluYWwiLCJhcHBpZCI6IjMwMjMyZDk1LTIzM2QtNDAzYi1hN2Q2LTRhZWY1NzQzOWY3MiIsImFwcGlkYWNyIjoiMSIsImlkcCI6Imh0dHBzOi8vc3RzLndpbmRvd3MubmV0LzRiMjQ2MmE0LWJiZWUtNDk1YS1hMGUxLWYyM2FlNTI0Y2M5Yy8iLCJpZHR5cCI6ImFwcCIsIm9pZCI6ImQ3Zjk5NjRiLWRiMjUtNGRmNi1hMjBjLTU1NjA5ZmFkMTk1OSIsInJoIjoiMC5BU3dBcEdJa1MtNjdXa21nNGZJNjVTVE1uQU1BQUFBQUFBQUF3QUFBQUFBQUFBQXNBQUEuIiwicm9sZXMiOlsiVGhyZWF0SW50ZWxsaWdlbmNlLlJlYWQuQWxsIl0sInN1YiI6ImQ3Zjk5NjRiLWRiMjUtNGRmNi1hMjBjLTU1NjA5ZmFkMTk1OSIsInRlbmFudF9yZ

_____________________________________________________________
# Start Calling Graph APIs with different parameters

In [12]:
# Prompt user to enter Domain name or IP Address to investigate
entity_ID = input("Enter the Domain or IP Address you want to investigate: ")

print("The Domain Name or IP Address to be investigated is: " + entity_ID)

The Domain Name or IP Address to be investigated is: 062fe.com


### Calling Graph API for Host Information 

In [13]:
import requests
import json

# Graph API base URL
graph_api_base_url = f"https://graph.microsoft.com/beta/security/threatIntelligence/hosts/{entity_ID}"

# Create headers with Authorization token
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

# Send GET request to Graph API endpoint
response = requests.get(graph_api_base_url, headers=headers)

# Check for HTTP errors
try:
    response.raise_for_status()
except requests.exceptions.HTTPError as http_err:
    print(f"HTTP error occurred: {http_err}")
except Exception as err:
    print(f"Other error occurred: {err}")
else:
    # Print response content as JSON
    print(json.dumps(json.loads(response.content), indent=4))


{
    "@odata.context": "https://graph.microsoft.com/beta/$metadata#security/threatIntelligence/hosts/$entity",
    "@odata.type": "#microsoft.graph.security.hostname",
    "id": "062fe.com",
    "firstSeenDateTime": "2013-11-30T04:26:32Z",
    "lastSeenDateTime": "2023-02-22T09:06:07Z",
    "registrar": "Guangdong JinWanBang Technology Investment Co., Ltd.",
    "registrant": "N/A"
}


### Calling Graph API Reputation 

In [6]:
import requests
import json

# Graph API base URL
graph_api_base_url = "https://graph.microsoft.com/beta/security/threatIntelligence/hosts/062fe.com/reputation"

# Create headers with Authorization token
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

# Send GET request to Graph API endpoint
response = requests.get(graph_api_base_url, headers=headers)

# Check for HTTP errors
try:
    response.raise_for_status()
except requests.exceptions.HTTPError as http_err:
    print(f"HTTP error occurred: {http_err}")
except Exception as err:
    print(f"Other error occurred: {err}")
else:
    # Print response content as JSON
    print(json.dumps(json.loads(response.content), indent=4))


{
    "@odata.context": "https://graph.microsoft.com/beta/$metadata#security/threatIntelligence/hosts('062fe.com')/reputation/$entity",
    "id": "MDYyZmUuY29t",
    "score": 100,
    "classification": "malicious",
    "rules": [
        {
            "name": "MDTI Intel Article",
            "description": "Threat Insights: DEV-1101 NakedPages adversary-in-the-middle phishing kit",
            "severity": "high",
            "relatedDetailsUrl": "https://ti.defender.microsoft.com/article/e0c5c210"
        },
        {
            "name": "Microsoft Blocklist (WatchList)",
            "description": "This indicator relates to an emerging threat actor DEV-0563",
            "severity": "high",
            "relatedDetailsUrl": null
        },
        {
            "name": "Microsoft Blocklist (WatchList)",
            "description": "Please read the linked article(s) for more information on this threat. [\"https://ti.defender.microsoft.com/article/e0c5c210\"]",
            "severity": "h

### Calling Graph API for Reputation - Return only Reputation Class, Score and Rules triggered

In [11]:
import requests
import json

# Graph API base URL
graph_api_base_url = "https://graph.microsoft.com/beta/security/threatIntelligence/hosts/062fe.com/reputation"

# Create headers with Authorization token
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

# Send GET request to Graph API endpoint
response = requests.get(graph_api_base_url, headers=headers)

# Check for HTTP errors
try:
    response.raise_for_status()
except requests.exceptions.HTTPError as http_err:
    print(f"HTTP error occurred: {http_err}")
except Exception as err:
    print(f"Other error occurred: {err}")
else:
    # Parse response content as JSON
    json_response = json.loads(response.content)
    
    # Check if the response contains the word "suspicious"
    is_suspicious = any("suspicious" in rule.get("classification", "") for rule in json_response.get("rules", []))
    
    # Print classification, score, and rules
    print(f"Classification: {json_response['classification']}")
    print(f"Score: {json_response['score']}")
    print("Rules:")
    for rule in json_response.get("rules", []):
        if is_suspicious:
            print(f"\x1b[41m\x1b[37mSUSPICIOUS SITE\x1b[0m: {rule.get('name', '')}")
        else:
            print(f"{rule.get('name', '')}")


Classification: malicious
Score: 100
Rules:
MDTI Intel Article
Microsoft Blocklist (WatchList)
Microsoft Blocklist (WatchList)
Name server


### Calling Graph API Components 

In [10]:
import requests
import json

# Graph API base URL
graph_api_base_url = "https://graph.microsoft.com/beta/security/threatIntelligence/hosts/062fe.com/components"

# Create headers with Authorization token
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

# Send GET request to Graph API endpoint
response = requests.get(graph_api_base_url, headers=headers)

# Check for HTTP errors
try:
    response.raise_for_status()
except requests.exceptions.HTTPError as http_err:
    print(f"HTTP error occurred: {http_err}")
except Exception as err:
    print(f"Other error occurred: {err}")
else:
    # Print response content as JSON
    print(json.dumps(json.loads(response.content), indent=4))


{
    "@odata.context": "https://graph.microsoft.com/beta/$metadata#Collection(microsoft.graph.security.hostComponent)",
    "value": [
        {
            "id": "RGlnaXRhbE9jZWFuJCQkJEhvc3RpbmcgUHJvdmlkZXIkJDA2MmZlLmNvbQ==",
            "firstSeenDateTime": "2023-02-09T14:07:37.8Z",
            "lastSeenDateTime": "2023-02-09T14:07:37.8Z",
            "name": "DigitalOcean",
            "version": "",
            "category": "Hosting Provider",
            "host": {
                "id": "062fe.com"
            }
        },
        {
            "id": "RmFjZWJvb2skJCQkVHJhY2tpbmcgUGl4ZWwkJDA2MmZlLmNvbQ==",
            "firstSeenDateTime": "2020-06-23T09:06:32.797Z",
            "lastSeenDateTime": "2020-06-23T09:06:38.148Z",
            "name": "Facebook",
            "version": "",
            "category": "Tracking Pixel",
            "host": {
                "id": "062fe.com"
            }
        },
        {
            "id": "R29vZ2xlIEFuYWx5dGljcyQkJCRUcmFja2luZyBQaXhlbCQkMDY

### Calling Graph API Cookies

In [4]:
import requests
import json

# Graph API base URL
graph_api_base_url = "https://graph.microsoft.com/beta/security/threatIntelligence/hosts/062fe.com/cookies"

# Create headers with Authorization token
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

# Send GET request to Graph API endpoint
response = requests.get(graph_api_base_url, headers=headers)

# Check for HTTP errors
try:
    response.raise_for_status()
except requests.exceptions.HTTPError as http_err:
    print(f"HTTP error occurred: {http_err}")
except Exception as err:
    print(f"Other error occurred: {err}")
else:
    # Print response content as JSON
    print(json.dumps(json.loads(response.content), indent=4))


{
    "@odata.context": "https://graph.microsoft.com/beta/$metadata#Collection(microsoft.graph.security.hostCookie)",
    "@odata.nextLink": "https://graph.microsoft.com/beta/security/threatIntelligence/hosts/062fe.com/cookies?$skip=25",
    "value": [
        {
            "id": "Lm1xbDUuY29tJCRsYW5nJCQwNjJmZS5jb20=",
            "firstSeenDateTime": "2020-06-23T09:06:32.797Z",
            "lastSeenDateTime": "2020-06-23T09:06:38.148Z",
            "domain": ".mql5.com",
            "name": "lang",
            "host": {
                "id": "062fe.com"
            }
        },
        {
            "id": "LjA2MmZlLmNvbSQkX19jZmR1aWQkJDA2MmZlLmNvbQ==",
            "firstSeenDateTime": "2020-06-23T09:05:18.351Z",
            "lastSeenDateTime": "2020-06-23T09:06:38.148Z",
            "domain": ".062fe.com",
            "name": "__cfduid",
            "host": {
                "id": "062fe.com"
            }
        },
        {
            "id": "MDYyZmUuY29tJCRQSFBTRVNTSUQkJDA2MmZlLmN

### Calling Graph API Trackers

In [5]:
import requests
import json

# Graph API base URL
graph_api_base_url = "https://graph.microsoft.com/beta/security/threatIntelligence/hosts/062fe.com/trackers?count=true"

# Create headers with Authorization token
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

# Send GET request to Graph API endpoint
response = requests.get(graph_api_base_url, headers=headers)

# Check for HTTP errors
try:
    response.raise_for_status()
except requests.exceptions.HTTPError as http_err:
    print(f"HTTP error occurred: {http_err}")
except Exception as err:
    print(f"Other error occurred: {err}")
else:
    # Print response content as JSON
    print(json.dumps(json.loads(response.content), indent=4))


{
    "@odata.context": "https://graph.microsoft.com/beta/$metadata#Collection(microsoft.graph.security.hostTracker)",
    "@odata.count": 5,
    "value": [
        {
            "id": "dWEtNzY1NTg2MjUkJEdvb2dsZUFuYWx5dGljc0FjY291bnROdW1iZXIkJDA2MmZlLmNvbQ==",
            "firstSeenDateTime": "2020-06-23T09:05:18.351Z",
            "lastSeenDateTime": "2020-06-23T09:05:18.393Z",
            "kind": "GoogleAnalyticsAccountNumber",
            "value": "ua-76558625",
            "host": {
                "id": "062fe.com"
            }
        },
        {
            "id": "dWEtNzY1NTg2MjUtMiQkR29vZ2xlQW5hbHl0aWNzVHJhY2tpbmdJZCQkMDYyZmUuY29t",
            "firstSeenDateTime": "2020-06-23T09:05:18.351Z",
            "lastSeenDateTime": "2020-06-23T09:05:18.393Z",
            "kind": "GoogleAnalyticsTrackingId",
            "value": "ua-76558625-2",
            "host": {
                "id": "062fe.com"
            }
        },
        {
            "id": "MTA2MDY3MjM2NTUzMTc2JCRGYWNlYm9

_____________________________________________________________
### Calling Graph API to get Article

#### Enter the Article ID for the article you wish to query for

In [ ]:
# Prompt user to enter article ID
articleId = input("Enter the article ID: ")

print(articleId)

#### Graph API Call for the Article Requested

In [7]:
import requests
import json

# Graph API base URL
graph_api_base_url = "https://graph.microsoft.com/beta/security/threatIntelligence/articles/fbbd8e0c"

# Create headers with Authorization token
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

# Send GET request to Graph API endpoint
response = requests.get(graph_api_base_url, headers=headers)

# Check for HTTP errors
try:
    response.raise_for_status()
except requests.exceptions.HTTPError as http_err:
    print(f"HTTP error occurred: {http_err}")
except Exception as err:
    print(f"Other error occurred: {err}")
else:
    # Print response content as JSON
    print(json.dumps(json.loads(response.content), indent=4))


{
    "@odata.context": "https://graph.microsoft.com/beta/$metadata#security/threatIntelligence/articles/$entity",
    "id": "fbbd8e0c",
    "createdDateTime": "2023-03-22T19:58:18.003Z",
    "lastUpdatedDateTime": "2023-04-06T19:00:02.147Z",
    "title": "IRIDIUM Uses TOR Hidden Services on Targets for Persistence and Evasion",
    "isFeatured": true,
    "tags": [
        "IRIDIUM",
        "T1546.008 - Accessibility Features",
        "T1569.002 - Service Execution",
        "T1133 - External Remote Services",
        "T1599 - Network Boundary Bridging",
        "T1572 - Protocol Tunneling"
    ],
    "imageUrl": "https://cdn-riq-ti.azureedge.net/Map.jpg",
    "summary": {
        "content": "Microsoft has identified a post-compromise persistence mechanism attributed to IRIDIUM that involves installing TOR hidden services on target devices for persistent access and network boundary protection evasion. This activity is being tracked as\u00a0*ShadowLink.*\r\n\r\nUpon accessing the tar

_____________________________________________________________
### Graph API call for Intel Profile

#### Get the Profile Id

In [ ]:
# Prompt user to enter article ID
ProfileId = input("Enter the Profile ID: ")

print(ProfileId)

#### Graph API Call to return Intel Profile Requested

In [8]:
import requests
import json

# Graph API base URL
graph_api_base_url = "https://graph.microsoft.com/beta/security/threatIntelligence/intelProfiles/9b01de37bf66d1760954a16dc2b52fed2a7bd4e093dfc8a4905e108e4843da80"

# Create headers with Authorization token
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

# Send GET request to Graph API endpoint
response = requests.get(graph_api_base_url, headers=headers)

# Check for HTTP errors
try:
    response.raise_for_status()
except requests.exceptions.HTTPError as http_err:
    print(f"HTTP error occurred: {http_err}")
except Exception as err:
    print(f"Other error occurred: {err}")
else:
    # Print response content as JSON
    print(json.dumps(json.loads(response.content), indent=4))


{
    "@odata.context": "https://graph.microsoft.com/beta/$metadata#security/threatIntelligence/intelProfiles/$entity",
    "id": "9b01de37bf66d1760954a16dc2b52fed2a7bd4e093dfc8a4905e108e4843da80",
    "kind": "actor",
    "title": "ACTINIUM",
    "firstActiveDateTime": "2020-02-24T11:13:16Z",
    "aliases": [
        "Primitive Bear",
        "SectorC08",
        "shuckworm",
        "Gamaredon",
        "UNC530",
        "Armageddon"
    ],
    "targets": [
        "Government Agencies & Services: Defense",
        "Government Agencies & Services: Law Enforcement",
        "Non-Government Organization: Human Rights Organization"
    ],
    "sponsorStates": [
        {
            "label": "Russia",
            "code": "ru"
        }
    ],
    "summary": {
        "content": "The actor that Microsoft tracks as ACTINIUM is a nation-state activity group based out of Russia. The Ukrainian government has publicly attributed this group to the Russian Federal Security Service (FSB). ACTINI

_____________________________________________________________
### Graph API Query for Vulnerability

#### Get the Vulnerability to query for

In [18]:
# Prompt user to enter article ID
VulnId = input("Enter the article ID: ")

print("The Vulnerability to be investigated is: " + VulnId)


The Vulnerability to be investigated is: CVE-2023-0669


#### API Call to return the Vulnerability information for request

In [16]:
import requests
import json

# Graph API base URL
graph_api_base_url = "https://graph.microsoft.com/beta/security/threatIntelligence/vulnerabilities/CVE-2023-0669"

# Create headers with Authorization token
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

# Send GET request to Graph API endpoint
response = requests.get(graph_api_base_url, headers=headers)

# Check for HTTP errors
try:
    response.raise_for_status()
except requests.exceptions.HTTPError as http_err:
    print(f"HTTP error occurred: {http_err}")
except Exception as err:
    print(f"Other error occurred: {err}")
else:
    # Print response content as JSON
    print(json.dumps(json.loads(response.content), indent=4))


{
    "@odata.context": "https://graph.microsoft.com/beta/$metadata#security/threatIntelligence/vulnerabilities/$entity",
    "id": "CVE-2023-0669",
    "createdDateTime": "2023-02-04T00:41:50.186Z",
    "lastModifiedDateTime": "2023-04-10T20:15:00Z",
    "publishedDateTime": "2023-02-06T20:15:00Z",
    "priorityScore": 86,
    "severity": "high",
    "commonWeaknessEnumerationIds": [
        "CWE-502"
    ],
    "hasChatter": true,
    "exploitsAvailable": true,
    "activeExploitsObserved": false,
    "description": {
        "content": "Fortra (formerly, HelpSystems) GoAnywhere MFT suffers from a pre-authentication command injection vulnerability in the License Response Servlet due to deserializing an arbitrary attacker-controlled object. This issue was patched in version 7.1.2.",
        "format": "markdown"
    },
    "remediation": {
        "content": null,
        "format": "markdown"
    },
    "cvss2Summary": {
        "score": 0.0,
        "severity": "critical",
        "ve

#### For trouble shooting, will return header information with the expected output

In [9]:
import requests
import json

# Graph API base URL
graph_api_base_url = "https://graph.microsoft.com/beta/security/threatIntelligence/hosts/062fe.com/reputation"

# Create headers with Authorization token
headers = {
    "Authorization": f"Bearer {access_token}",
    "Content-Type": "application/json"
}

# Send GET request to Graph API endpoint
response = requests.get(graph_api_base_url, headers=headers)

# Check for HTTP errors
try:
    response.raise_for_status()
except requests.exceptions.HTTPError as http_err:
    print(f"HTTP error occurred: {http_err}")
except Exception as err:
    print(f"Other error occurred: {err}")
else:
    # Print response headers
    print("Response Headers:")
    print("-----------------")
    for header, value in response.headers.items():
        print(f"{header}: {value}")
    print()

    # Print response content as JSON
    print("Response Content:")
    print("-----------------")
    print(json.dumps(json.loads(response.content), indent=4))


Response Headers:
-----------------
Cache-Control: no-store, must-revalidate, no-cache, max-age=0
Pragma: no-cache
Transfer-Encoding: chunked
Content-Type: application/json;odata.metadata=minimal;odata.streaming=true;IEEE754Compatible=false;charset=utf-8
Content-Encoding: gzip
Expires: 0
Strict-Transport-Security: max-age=31536000
request-id: 87560e26-fa77-468a-8600-504f20dc348b
client-request-id: 87560e26-fa77-468a-8600-504f20dc348b
x-ms-ags-diagnostic: {"ServerInfo":{"DataCenter":"North Central US","Slice":"E","Ring":"3","ScaleUnit":"001","RoleInstance":"CH01EPF0000D455"}}
X-Content-Type-Options: nosniff
OData-Version: 4.0
Vary: Origin,Access-Control-Request-Method,Access-Control-Request-Headers, Accept-Encoding
Date: Fri, 14 Apr 2023 13:17:52 GMT

Response Content:
-----------------
{
    "@odata.context": "https://graph.microsoft.com/beta/$metadata#security/threatIntelligence/hosts('062fe.com')/reputation/$entity",
    "id": "MDYyZmUuY29t",
    "score": 100,
    "classification": "